In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pystan
import seaborn as sns

import pandas as pd
from airsenal.framework.utils import *

In [ ]:
model = pystan.StanModel(file="dynamic_model.stan")

In [ ]:
rows = []

results = session.query(Result).all()
for result in results:
    row = {}
    row['gameweek'] = result.fixture.gameweek
    row['season'] = result.fixture.season
    row['home_team'] = result.fixture.home_team
    row['away_team'] = result.fixture.away_team
    row['home_goals'] = result.home_score
    row['away_goals'] = result.away_score
    rows.append(row)

df = pd.DataFrame(rows)
df = df.loc[df["season"].isin(["1819", "1718", "1617"])]

df.loc[df["season"] == "1718", "gameweek"] = df.loc[df["season"] == "1718", "gameweek"] + 38
df.loc[df["season"] == "1819", "gameweek"] = df.loc[df["season"] == "1819", "gameweek"] + 2 * 38

teams = list(set(df["home_team"]).union(set(df["away_team"])))
team_indices = {team: i + 1 for i, team in enumerate(teams)}

df = df.replace(team_indices)

In [ ]:
stan_data = {
    "gameweek": df["gameweek"].values,
    "home_team": df["home_team"].values,
    "away_team": df["away_team"].values,
    "home_goals": df["home_goals"].values,
    "away_goals": df["away_goals"].values,
    "n_match": len(df),
    "n_team": len(teams),
    "n_gameweeks": df["gameweek"].nunique()
}

In [ ]:
fit = model.sampling(stan_data, iter=1000, control={"adapt_delta": 0.9, "max_treedepth": 20})

In [ ]:
def plot_team_ability(team_name, ax=None, param="a"):
    team_idx = team_indices[team_name] - 1
    if ax is None:
        fig, ax = plt.subplots()
    ax.fill_between(
        range(114),
        np.percentile(fit[param][:, team_idx, :], 16, axis=0),
        np.percentile(fit[param][:, team_idx, :], 84, axis=0),
        color="0.5",
        alpha=0.5
    )
    ax.plot(range(114), fit[param][:, team_idx, :].mean(axis=0), c="k")
    return ax

In [ ]:
plot_team_ability("MCI")

In [ ]:
plot_team_ability("MCI", param="b")